In [1]:
#export
import sys
if __name__ == '__main__': sys.path.append('..')
import download_youtube_subtitle.common as common
from pprint import pprint
def pj(*args, **kargs):
    if common.IN_JUPYTER:
        pprint(*args, **kargs)
        
from functools import partial
import sys

perr = partial(print, "ERR: ")

In [2]:
#export
import requests
import socket
socket.setdefaulttimeout(5.)

In [3]:
#export
from pytube.innertube import InnerTube
def get_tracks_title(videoID):
    innertube = InnerTube()
    video_info = innertube.player(videoID)
    return video_info["captions"]["playerCaptionsTracklistRenderer"]["captionTracks"], video_info["videoDetails"]["title"] 

In [4]:
_captions,_title =get_tracks_title("wgNiGj1nGYE")
print(_title, _captions[:2])

Version 1.5 Special Program｜Genshin Impact [{'baseUrl': 'https://www.youtube.com/api/timedtext?v=wgNiGj1nGYE&asr_langs=de,en,es,fr,id,it,ja,ko,nl,pt,ru,tr,vi&caps=asr&exp=xftt,xctw&xoaf=5&hl=en&ip=0.0.0.0&ipbits=0&expire=1631358644&sparams=ip,ipbits,expire,v,asr_langs,caps,exp,xoaf&signature=ED4AEA39C4FA1BD9F3B4079B94D18CA28981D551.B5A147A4A495018453BB5405B0125D4989EE70A6&key=yt8&lang=zh-Hans&fmt=srv3', 'name': {'runs': [{'text': 'Chinese (Simplified)'}]}, 'vssId': '.zh-Hans', 'languageCode': 'zh-Hans', 'isTranslatable': True}, {'baseUrl': 'https://www.youtube.com/api/timedtext?v=wgNiGj1nGYE&asr_langs=de,en,es,fr,id,it,ja,ko,nl,pt,ru,tr,vi&caps=asr&exp=xftt,xctw&xoaf=5&hl=en&ip=0.0.0.0&ipbits=0&expire=1631358644&sparams=ip,ipbits,expire,v,asr_langs,caps,exp,xoaf&signature=ED4AEA39C4FA1BD9F3B4079B94D18CA28981D551.B5A147A4A495018453BB5405B0125D4989EE70A6&key=yt8&lang=zh-Hant&fmt=srv3', 'name': {'runs': [{'text': 'Chinese (Traditional)'}]}, 'vssId': '.zh-Hant', 'languageCode': 'zh-Hant', 

In [5]:
#export
# add tlang=zh-Hans to baseUrl
# will get translation

# getting track info

class CaptionNotAvailableException(Exception):
    pass 
import re
import json
import urllib

In [6]:
#export
# dealing with transcript
import math
from functools import partial
import sys
from xml.dom.minidom import parseString
import html
def parseTranscript(transcript, remove_font_tag=True):
    try:
        dom = parseString(transcript.text)
    except :
        perr("check your lang code")
        perr("server response")
        perr(transcript.text)
        exit(1)
    texts = dom.getElementsByTagName('p')
    
    _eachTxt = partial(eachTxt, remove_font_tag=remove_font_tag)
    texts = list(map( _eachTxt, texts,))
    return texts 

def each_sent(o, file=sys.stdout):
    if len(o['text'].strip()) == 0 and len(o.get('translate_text', "").strip()) == 0:
        return
    start = o['start']
    start = float(start)
    start /= 1000
    minute = math.floor(start/60)
    second = math.floor(start%60)
    p = partial(print, file=file)
    p(f"---------{minute:02d}:{second:02d}----------")
    p(o['text'])
    translate_text = o.get('translate_text', None)
    if translate_text:
        p(translate_text)
   

In [7]:
#export
# dealing with valid filename
# https://github.com/django/django/blob/master/django/utils/text.py
import re
def get_valid_filename(s):
    """
    Return the given string converted to a string that can be used for a clean
    filename. Remove leading and trailing spaces; convert other spaces to
    underscores; and remove anything that is not an alphanumeric, dash,
    underscore, or dot.
    >>> get_valid_filename("john's portrait in 2004.jpg")
    'johns_portrait_in_2004.jpg'
    """
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

In [8]:
#export
import copy
def merge_subtitle(subtitle, subtitle_cn):
    """
    merge subtitle_cn(traslation) to subtitle(origin).
    cc and translated cc aren't always align, 
    I found at least in cn and ja, translated cc are less than cc
    see  videoID='HSz7Q4YnQ_M'
    cc and translated cc aren't always equal in time see wgNiGj1nGYE for translation ja
    """
    subtitle = copy.deepcopy(subtitle) # original transcript
    subtitle_cn = copy.deepcopy(subtitle_cn) # translation script
    
    TRANSLATE_TEXT="translate_text"
    TEXT="text"
    START="start"
    
    # NOTE not sure how to merge them properly
    
    # indexer for subtitle 
    sub_p = 0
    sub_p_cn = 0
    
    while sub_p < len(subtitle) and sub_p_cn < len(subtitle_cn):
        
        sub = subtitle[sub_p]
        sub_cn = subtitle_cn[sub_p_cn]
        
        if TRANSLATE_TEXT not in sub: sub[TRANSLATE_TEXT] = ""
        
        if float(sub[START]) >= float(sub_cn[START]) :
        # sub goes first, being chased by sub_cn
        
            # for separating the sentence
            if len(sub[TRANSLATE_TEXT]) != 0: sub[TRANSLATE_TEXT] += "\n"
            
            sub[TRANSLATE_TEXT] +=  sub_cn[TEXT]
            
            sub_p_cn += 1
            
        else :
            sub_p += 1
            
    # add empty field for subitle
    while sub_p < len(subtitle):
        assert sub_p_cn == len(subtitle_cn)
        
        sub = subtitle[sub_p]
        if TRANSLATE_TEXT not in sub: sub[TRANSLATE_TEXT] = ""
        sub_p += 1
        
    # add the rest of subtitle_cn to the last one of subtitle
    while sub_p_cn < len(subtitle_cn):
        assert sub_p == len(subtitle)
        
        sub = subtitle[-1]
        
        if TRANSLATE_TEXT not in sub: sub[TRANSLATE_TEXT] = ""
            
        if len(sub[TRANSLATE_TEXT]) != 0: sub[TRANSLATE_TEXT] += "\n"
        
        sub[TRANSLATE_TEXT] += sub_cn[TEXT]
        sub_p_cn += 1
        
    assert sub_p == len(subtitle)
    assert sub_p_cn == len(subtitle_cn)
        
    return subtitle

In [9]:
#export
# dealing with xml.dom
import re
import xml
def eachTxt(txt, remove_font_tag):
    start = txt.attributes.get("t").value
    dur = '0' if txt.attributes.get("d") is None \
        else txt.attributes.get("d").value 
    if txt.firstChild is None:
        txt = ""
    elif isinstance(txt.firstChild, xml.dom.minidom.Text):
        txt = txt.firstChild.data
    elif txt.firstChild.tagName == 's':
        ret_txt = ""
        for sTxt in txt.childNodes:
            if sTxt.firstChild: ret_txt += sTxt.firstChild.data
        txt = ret_txt
    
    
    txt = html.unescape((txt))
    if remove_font_tag:
        txt =   re.sub(r'</?font[^>]*>','', txt)
        
    return {
        "start":start, 
        "dur": dur,
        "text": txt
    }

In [10]:
#test_export
# here to check parsing dom
videoID="SVdQ-2njL6U" # test [Fix issue where translation is not True by marosoft · Pull Request #3 · xsthunder/download-youtube-subtitle](https://github.com/xsthunder/download-youtube-subtitle/pull/3)
captionTracks, title = get_tracks_title(videoID)
baseUrl = captionTracks[0]['baseUrl']
transcript = requests.get(baseUrl)
dom = parseString(transcript.text)
texts = dom.getElementsByTagName('p')
_eachTxt = partial(eachTxt, remove_font_tag=True)
for i,t in enumerate(texts):
    if i>5:break
    print(t, _eachTxt(t))

<DOM Element: p at 0x7f0930079178> {'start': '140', 'dur': '4570', 'text': 'hi this is Louis from Rossville repair'}
<DOM Element: p at 0x7f0930079638> {'start': '2960', 'dur': '1750', 'text': '\n'}
<DOM Element: p at 0x7f09300796d0> {'start': '2970', 'dur': '3869', 'text': 'group in this video in a couple of'}
<DOM Element: p at 0x7f0930079c28> {'start': '4700', 'dur': '2139', 'text': '\n'}
<DOM Element: p at 0x7f0930079cc0> {'start': '4710', 'dur': '4469', 'text': "minutes you're going to watch me as I"}
<DOM Element: p at 0x7f091bfc22a8> {'start': '6829', 'dur': '2350', 'text': '\n'}


In [11]:
_eachTxt(texts[1])['text'].strip()

''

In [12]:
o = _eachTxt(texts[1])
len(o['text'].strip()) == 0 and len(o.get('translate_text', "").strip())

0

In [13]:
print(dom.toprettyxml()[:1000])

<?xml version="1.0" ?>
<timedtext format="3">
	

	<head>
		

		<ws id="0"/>
		

		<ws id="1" ju="0" mh="2" sd="3"/>
		

		<wp id="0"/>
		

		<wp ah="20" ap="6" av="100" cc="40" id="1" rc="2"/>
		

	</head>
	

	<body>
		

		<w id="1" t="0" wp="1" ws="1"/>
		

		<p d="4570" t="140" w="1">
			<s ac="252">hi</s>
			<s ac="201" t="1000"> this</s>
			<s ac="201" t="1480"> is</s>
			<s ac="146" t="1540"> Louis</s>
			<s ac="245" t="1989"> from</s>
			<s ac="217" t="2200"> Rossville</s>
			<s ac="227" t="2410"> repair</s>
		</p>
		

		<p a="1" d="1750" t="2960" w="1">
</p>
		

		<p d="3869" t="2970" w="1">
			<s ac="239">group</s>
			<s ac="187" t="179"> in</s>
			<s ac="252" t="360"> this</s>
			<s ac="208" t="810"> video</s>
			<s ac="235" t="1110"> in</s>
			<s ac="252" t="1349"> a</s>
			<s ac="157" t="1440"> couple</s>
			<s ac="252" t="1710"> of</s>
		</p>
		

		<p a="1" d="2139" t="4700" w="1">
</p>
		

		<p d="4469" t="4710" w="1">
			<s ac="236">minutes</s>
			<s ac="252" t="270"> you

In [14]:
#test_export
#here to break up the procss
# videoID="Zd14s2WW-Tc"
videoID="wgNiGj1nGYE"
captionTracks, title = get_tracks_title(videoID)
baseUrl = captionTracks[0]['baseUrl']
transcript = requests.get(baseUrl)
subtitle = parseTranscript(transcript)
baseUrl = captionTracks[0]['baseUrl'] + '&tlang=ja'
transcript = requests.get(baseUrl)
subtitle_cn = parseTranscript(transcript)
merged_subtitle = merge_subtitle(subtitle, subtitle_cn)
merged_subtitle[:4]

[{'start': '0',
  'dur': '5056',
  'text': '从前，有一对双胞胎结伴在宇宙中旅行',
  'translate_text': '昔々、宇宙を一緒に旅している双子のペアがいました'},
 {'start': '5398',
  'dur': '2398',
  'text': '但有一天，他们前路遇阻',
  'translate_text': 'しかしある日、彼らの道は封鎖されました'},
 {'start': '7988',
  'dur': '2801',
  'text': '被一个未知的神明生生分离',
  'translate_text': '未知の神によって隔てられている'},
 {'start': '11745', 'dur': '804', 'text': '空！', 'translate_text': 'ヌル！'}]

In [15]:
print(transcript.text[:1000])

<?xml version="1.0" encoding="utf-8" ?><timedtext format="3">
<body>
<p t="0" d="5056">昔々、宇宙を一緒に旅している双子のペアがいました</p>
<p t="5398" d="2398">しかしある日、彼らの道は封鎖されました</p>
<p t="7988" d="2801">未知の神によって隔てられている</p>
<p t="11745" d="804">ヌル！</p>
<p t="12971" d="882">蛍光！</p>
<p t="14404" d="3046">そして今、2つはついに再会しました...</p>
<p t="17872" d="4474">元神バージョン1.5の特別プログラムで！</p>
<p t="22400" d="2905">はい！ハハ……</p>
<p t="25305" d="1920">みなさん、ようこそ！私はザック・アギラールです</p>
<p t="27225" d="3254">元の神の男性旅行者のための空の声</p>
<p t="30682" d="2296">みなさん、こんにちは。サラミラークルーズです。</p>
<p t="32978" d="2312">私は女性旅行者のインを吹き替えました。</p>
<p t="35784" d="2216">このショーに一緒に参加できるのは素晴らしいことです</p>
<p t="38000" d="4083">特に私たちのキャラクターがバージョン1.4でようやく出会ったからです</p>
<p t="42178" d="1722">はい</p>
<p t="43900" d="2712">彼らが私をこのショーに招待してくれてうれしい</p>
<p t="46688" d="3216">私はいつも特別なプログラムがどのように見えるかについて興味がありました</p>
<p t="49971" d="3815">あなたとコリーナは前のショーでとても幸せでした</p>
<p t="53939" d="3006">ねえ、これがスペシャルショーの本質です！</p>
<p t="57136" d="3387">もちろん、今日も例外はありません。</p>
<p t="60599" d="1265">キースシルバー

In [16]:
#export
def format_caption(caption):
    ret = f"{caption['vssId']:8s} {caption['name']['runs'][0]['text']}"
    if 'a.' in caption['vssId']:
        ret += ' automatically generated by youtube'
    return ret

In [17]:
print(captionTracks[1]['name'])
format_caption(captionTracks[1])

{'runs': [{'text': 'Chinese (Traditional)'}]}


'.zh-Hant Chinese (Traditional)'

In [18]:
#test_export
#here test output
output_file = get_valid_filename(f'{title}.txt')
with open(output_file , 'w', encoding='UTF-8') as f:
    print("save to ", output_file)
    for sent in subtitle:
        each_sent(sent, file=f)

save to  Version_1.5_Special_ProgramGenshin_Impact.txt


In [19]:
#export
def parseVideoID(videoID):
    if 'youtu' in videoID:
        videoID = re.search('v=([^&]+)', videoID).group(1)
        
    video_link = f'https://www.youtube.com/watch?v={videoID}'
    return videoID, video_link

In [20]:
#test_export
videoID = 'https://www.youtube.com/watch?v=wgNiGj1nGYE'
videoID, video_link= parseVideoID(videoID)
videoID = 'wgNiGj1nGYE'
assert (videoID, video_link) == parseVideoID(videoID)

In [21]:
#export
import fire
import sys
from functools import partial
import json
import re
from typing import Union, Optional
def main(
    videoID:str, 
    translation:Union[str,bool]='zh-Hans', 
    caption_num:int=0, 
    caption_num_second:int=None,
    output_file:str=None, 
    save_to_file:bool=True, 
    to_json:bool=False, 
    remove_font_tag:bool=True, 
    ):
    """
    download youtube closed caption(subtitles) by videoID
    
    Examples:
    dl-youtube-cc -h # to see this helpful infomation
    dl-youtube-cc wgNiGj1nGYE --translation 'ja' # use japanese translation, see ./lang_code for full list
    dl-youtube-cc wgNiGj1nGYE --caption_num=1 --translation 'ja' # choose the caption num for original transcript and use japanese translation,
    dl-youtube-cc wgNiGj1nGYE --caption_num=1 --caption_num_second=2 # manually choose the original and translation transcript from available caption list
    dl-youtube-cc wgNiGj1nGYE --translation False # without translation
    dl-youtube-cc wgNiGj1nGYE --save_to_file=False # print stuff in console
    dl-youtube-cc wgNiGj1nGYE --output_file='test.txt' # print stuff in named file
    dl-youtube-cc wgNiGj1nGYE --to_json=True # print stuff in json
    
    Argument:
    videoID : the video link or the id of youtube video, the string after 'v=' in a youtube video link
    translation : which will be displayed as original transcript, default to 'zh-Hans' for simplified Chinese, see ./lang_code.json for full list, or pass False to disable translation
    caption_num : choose the caption which will be displayed as original transcript
    caption_num_second : will surpass translation option, choose the caption which will be displayed as translation transcript
    output_file : default to video title
    save_to_file : pass False to print in console
    to_json: pass True to export caption to json
    remove_font_tag: remove font tag
    """    
    videoID, video_link, = parseVideoID(videoID)
    captionTracks, title = None, None
    try:
        captionTracks, title = get_tracks_title(videoID)
    except: 
        perr("can't not retrive caption, ", " for detail")
        raise 
        
    
    info = partial(print, "INFO: ")
    
    info("available caption(s):")
    for i, caption in enumerate(captionTracks):
        mark = '⭕'
        if caption_num == i:mark = '✔ as original'
        if caption_num_second == i: mark = '✔ as translation' 
        notice = f"#{i:<2} {mark}"
        info(notice, format_caption(caption))
    info("given by --caption_num default to 0 as original")
    if caption_num_second is None: info("you may use --caption_num_second intead of --translation to spefify translation transript")
    get_caption_url = lambda i:captionTracks[i]["baseUrl"]
    
    # as original caption
    caption = captionTracks[caption_num]
    baseUrl = caption['baseUrl']
    transcript = requests.get(baseUrl)
#     info('using',f"{caption_num}.", format_caption(caption))
    
    _parseTranscript = partial(parseTranscript, remove_font_tag=remove_font_tag)
    subtitle = _parseTranscript(transcript, )
    sent_subtitle = subtitle
    
    output_json = { "original": subtitle }
    
    if translation or caption_num_second:
        if translation: baseUrl = caption['baseUrl'] + '&tlang=' + translation
        if caption_num_second: 
            translation = False
            baseUrl = get_caption_url(caption_num_second)
        transcript = requests.get(baseUrl)
        subtitle_cn = _parseTranscript(transcript)
        merged_subtitle = merge_subtitle(subtitle, subtitle_cn)
        sent_subtitle = merged_subtitle
        output_json = {
                 "original": subtitle,
                 "translation":subtitle_cn,
                # note that it's not guaranteed to be aligned.
                "merged": merged_subtitle,
            }
        
    ######################## save to file
            
    f = sys.stdout
    if save_to_file :
        if output_file is None: 
            if to_json:
                output_file = get_valid_filename(f'{title}.json')
            else:
                output_file = get_valid_filename(f'{title}.txt')
        f = open(output_file , 'w', encoding='UTF-8')
        info("Save to ", output_file )
    
    if to_json:
        json.dump(
            output_json
             , f, indent=4, ensure_ascii=False)
        return 
    
    pfile = partial(print, file=f)
    pfile("video_link\t", video_link)
    pfile("original\t", format_caption(captionTracks[caption_num]))
    if translation: pfile("translation\t",translation)
    if caption_num_second: pfile("translation\t", format_caption(captionTracks[caption_num_second]))
    for sent in sent_subtitle:
        each_sent(sent, file=f)
        pfile()

In [22]:
#export
from functools import partial
def set_fire(fn):
    if common.IN_TRAVIS or common.IN_JUPYTER:
        return 
    fire.Fire(fn)
if __name__ == '__main__':
    if common.IN_TRAVIS or common.IN_JUPYTER:
        pass
    else :
        set_fire(main)
fire_main = partial(set_fire, main)

In [23]:
#test_export
# see [Fix issue where translation is not True by marosoft · Pull Request #3 · xsthunder/download-youtube-subtitle](https://github.com/xsthunder/download-youtube-subtitle/pull/3)
main("SVdQ-2njL6U", translation=False, output_file=f'a.txt')

INFO:  available caption(s):
INFO:  #0  ✔ as original a.en     English (auto-generated) automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  a.txt


In [24]:
#test_export
main("wgNiGj1nGYE", translation='ja')

INFO:  available caption(s):
INFO:  #0  ✔ as original .zh-Hans Chinese (Simplified)
INFO:  #1  ⭕ .zh-Hant Chinese (Traditional)
INFO:  #2  ⭕ .en-US   English (United States)
INFO:  #3  ⭕ .fr      French
INFO:  #4  ⭕ .de      German
INFO:  #5  ⭕ .id      Indonesian
INFO:  #6  ⭕ .pt      Portuguese
INFO:  #7  ⭕ .ru      Russian
INFO:  #8  ⭕ .es      Spanish
INFO:  #9  ⭕ .th      Thai
INFO:  #10 ⭕ .vi      Vietnamese
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  Version_1.5_Special_ProgramGenshin_Impact.txt


In [25]:
print(f"{2:<2}", 1)

2  1


In [26]:
#test_export
main("wgNiGj1nGYE", caption_num=6, caption_num_second=0)

INFO:  available caption(s):
INFO:  #0  ✔ as translation .zh-Hans Chinese (Simplified)
INFO:  #1  ⭕ .zh-Hant Chinese (Traditional)
INFO:  #2  ⭕ .en-US   English (United States)
INFO:  #3  ⭕ .fr      French
INFO:  #4  ⭕ .de      German
INFO:  #5  ⭕ .id      Indonesian
INFO:  #6  ✔ as original .pt      Portuguese
INFO:  #7  ⭕ .ru      Russian
INFO:  #8  ⭕ .es      Spanish
INFO:  #9  ⭕ .th      Thai
INFO:  #10 ⭕ .vi      Vietnamese
INFO:  given by --caption_num default to 0 as original
INFO:  Save to  Version_1.5_Special_ProgramGenshin_Impact.txt


In [27]:
#test_export
main("wgNiGj1nGYE", caption_num=0, caption_num_second=3, output_file="0,3-zh,es.txt")

INFO:  available caption(s):
INFO:  #0  ✔ as original .zh-Hans Chinese (Simplified)
INFO:  #1  ⭕ .zh-Hant Chinese (Traditional)
INFO:  #2  ⭕ .en-US   English (United States)
INFO:  #3  ✔ as translation .fr      French
INFO:  #4  ⭕ .de      German
INFO:  #5  ⭕ .id      Indonesian
INFO:  #6  ⭕ .pt      Portuguese
INFO:  #7  ⭕ .ru      Russian
INFO:  #8  ⭕ .es      Spanish
INFO:  #9  ⭕ .th      Thai
INFO:  #10 ⭕ .vi      Vietnamese
INFO:  given by --caption_num default to 0 as original
INFO:  Save to  0,3-zh,es.txt


In [28]:
#test_export
main("wgNiGj1nGYE", caption_num=6, translation="ja", to_json=True)

INFO:  available caption(s):
INFO:  #0  ⭕ .zh-Hans Chinese (Simplified)
INFO:  #1  ⭕ .zh-Hant Chinese (Traditional)
INFO:  #2  ⭕ .en-US   English (United States)
INFO:  #3  ⭕ .fr      French
INFO:  #4  ⭕ .de      German
INFO:  #5  ⭕ .id      Indonesian
INFO:  #6  ✔ as original .pt      Portuguese
INFO:  #7  ⭕ .ru      Russian
INFO:  #8  ⭕ .es      Spanish
INFO:  #9  ⭕ .th      Thai
INFO:  #10 ⭕ .vi      Vietnamese
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  Version_1.5_Special_ProgramGenshin_Impact.json


In [29]:
#test_export
#fix eachTxt, allow txt.firstChild = None
main('https://www.youtube.com/watch?v=Zd14s2WW-Tc', caption_num=1)

INFO:  available caption(s):
INFO:  #0  ⭕ a.en     English (auto-generated) automatically generated by youtube
INFO:  #1  ✔ as original .en.j3PyPqV-e1s English - CC (English)
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  Chinese_Politics__Authoritarian_Regimes__Professor_Rory_Truex__Talks_at_Google.txt


In [30]:
#test_export
import sure
main('https://www.youtube.com/watch?v=EozTm6ZVf1U', caption_num=1)

INFO:  available caption(s):
INFO:  #0  ⭕ a.en     English (auto-generated) automatically generated by youtube
INFO:  #1  ✔ as original .en-US   English (United States)
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  Quarantine_Dispatch_Trump_Gets_Serious__NYC_Shuts_Down__The_Daily_Show.txt


In [31]:
#test_export
main('https://www.youtube.com/watch?v=EozTm6ZVf1U', caption_num=1)

INFO:  available caption(s):
INFO:  #0  ⭕ a.en     English (auto-generated) automatically generated by youtube
INFO:  #1  ✔ as original .en-US   English (United States)
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  Quarantine_Dispatch_Trump_Gets_Serious__NYC_Shuts_Down__The_Daily_Show.txt


In [32]:
#test_export
main('MhCEdIqFCck') # fix title with quotes "

INFO:  available caption(s):
INFO:  #0  ✔ as original .en      English
INFO:  #1  ⭕ a.en     English (auto-generated) automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  Dont_Lose_Your_Accent__Learning_Accents_-_TREVOR_NOAH_Pay_Back_The_Funny.txt


In [33]:
#test_export
main('HSz7Q4YnQ_M') # fix when cc's length and translated cc's don't match

INFO:  available caption(s):
INFO:  #0  ✔ as original a.en     English (auto-generated) automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  Silent_Voice_Koe_no_Katachi_Yamada_and_the_Unspoken.txt


In [34]:
#test_export
def read_all_content(f):
    with open(f, 'r', encoding='UTF-8') as f:
        ret = ''.join(f.readlines())
    return ret

In [35]:
#test_export
f = 'no_font_tag.txt'
main('tktbVrTFUkc', output_file=f) # remove font tag
assert '<font' not in read_all_content(f)
assert '</font>' not in read_all_content(f)
del f

INFO:  available caption(s):
INFO:  #0  ✔ as original a.en     English (auto-generated) automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  no_font_tag.txt


In [36]:
#test_export
f = 'save_font_tag.txt'
main('tktbVrTFUkc', remove_font_tag=False, output_file=f) # remove font tag
del f

INFO:  available caption(s):
INFO:  #0  ✔ as original a.en     English (auto-generated) automatically generated by youtube
INFO:  given by --caption_num default to 0 as original
INFO:  you may use --caption_num_second intead of --translation to spefify translation transript
INFO:  Save to  save_font_tag.txt


In [37]:
common.save_and_export_notebook('main')

<IPython.core.display.Javascript object>

Converted main.ipynb to ../download_youtube_subtitle/main.py
Converted main.ipynb to ../tests/test_main.py


<IPython.core.display.Javascript object>